In [2]:
pip install inference_sdk


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from inference_sdk import InferenceHTTPClient

import cv2
from PIL import Image
import pytesseract
import fitz

In [4]:
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="BIrE2FC1DoCP2tdXB9MA"
)

# Define the model ID
model_id = "resume-parser-bchlq/1"


In [5]:
def pdf_to_jpg(pdf_path):
    pdf_document = fitz.open(pdf_path)
    for page_number in range(len(pdf_document)):
        page = pdf_document.load_page(page_number)
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        output_path = f"cv_{page_number + 1}.jpg"
        img.save(output_path, "JPEG")
        return output_path


In [6]:

def extract_skills(image_path):
    image_path=pdf_to_jpg(image_path)
    # Extract the 'Skills' section from the result
    CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="BIrE2FC1DoCP2tdXB9MA"
)

# Define the model ID

    model_id = "resume-parser-bchlq/1"

    result = CLIENT.infer(image_path, model_id=model_id)

    image=cv2.imread(image_path)
    skills_section = None
    for prediction in result['predictions']:
        if prediction['class'] == 'skills':
            x = int(prediction['x'] - prediction['width'] / 2)
            y = int(prediction['y'] - prediction['height'] / 2)
            width = int(prediction['width'])
            height = int(prediction['height'])

        # Crop the skills section from the image
        skills_image = image[y:y+height, x:x+width]

        # Convert the image to RGB (needed for Tesseract)
        skills_image_rgb = cv2.cvtColor(skills_image, cv2.COLOR_BGR2RGB)
        
        # Convert to PIL Image for Tesseract
        pil_image = Image.fromarray(skills_image_rgb)
        
        # Use Tesseract to extract text
        skills_text = pytesseract.image_to_string(pil_image)

        # Print the extracted skills section text
        print("Extracted Skills Section:")
        print(skills_text)
        break
    else:
        print("No 'Skills' section found in the image.")


In [7]:
extract_skills("cv5.jpeg")

Extracted Skills Section:
SKILLS

Machine Learning

Ansible, Apache Airflow

Deep Learning (NLP, CNN, RAN)
Hadoop, Hbase, Spark

Proxmox, VMware, VitualBor, Docker
MySQL, PostgreSQL, MongoD8, Hive
Python, Java, , Seala

Hum, C85, JavaScript, PHP, XML
Android Studio



In [14]:
def get_name(image_path):
    # Convert PDF to JPG
    image_path = pdf_to_jpg(image_path)

    # Initialize the Roboflow inference client
    CLIENT = InferenceHTTPClient(
        api_url="https://detect.roboflow.com",
        api_key="BIrE2FC1DoCP2tdXB9MA"
    )

    # Define the model ID
    model_id = "resume-parser-bchlq/1"

    # Perform inference
    result = CLIENT.infer(image_path, model_id=model_id)

    # Read the image
    image = cv2.imread(image_path)
    name_section = None

    # Iterate over predictions to find 'Name'
    for prediction in result['predictions']:
        if prediction['class'] == 'Name':
            x = int(prediction['x'] - prediction['width'] / 2)
            y = int(prediction['y'] - prediction['height'] / 2)
            width = int(prediction['width'])
            height = int(prediction['height'])

            # Crop the name section from the image
            name_image = image[y:y+height, x:x+width]

            # Convert the image to RGB (needed for Tesseract)
            name_image_rgb = cv2.cvtColor(name_image, cv2.COLOR_BGR2RGB)

            # Convert to PIL Image for Tesseract
            pil_image = Image.fromarray(name_image_rgb)

            # Use Tesseract to extract text
            name_text = pytesseract.image_to_string(pil_image)

            # Print the extracted name
            print("Name is:")
            return name_text
    else:
        print("No 'Name' section found")
        return None


In [16]:
print(get_name('cv-1.pdf'))

Name is:
Hassan BARKA BICHARA

